In [2]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv
import os
import json
from copy import deepcopy

In [1]:
from transformers import BertModel, BertTokenizer

g:\Agentic-Coffee-Shop-Chatbot\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [18]:
client=ChatGroq(api_key=api_key,model=model_name)

In [43]:
def get_response(messages):
        messages=deepcopy(messages)

        system_prompt="""
            You are a helpful AI assistant for a coffee shop application which serves drinks and pastries.
            Your task is to determine whether the user is asking something relevant to the coffee shop or not.
            The user is allowed to:
            1. Ask questions about the coffee shop, like location, working hours, menue items and coffee shop related questions.
            2. Ask questions about menue items, they can ask for ingredients in an item and more details about the item.
            3. Make an order.
            4. ASk about recommendations of what to buy.

            The user is NOT allowed to:
            1. Ask questions about anything else other than our coffee shop.
            2. Ask questions about the staff or how to make a certain menue item.

            Your output must be in a valid JSON format.
            Each key and value must be wrapped in double quotes.
            Do not include any extra text or explanation outside the JSON.
            Strictly return only the JSON object.
            {
            "chain of thought": go over each of the points above and make see if the message lies under this point or not. Then you write some your thoughts about what point is this input relevant to.
            "decision": "allowed" or "not allowed". Pick one of those. and only write the word.
            "message": leave the message empty if it's allowed, otherwise write "Sorry, I can't help with that. Can I help you with your order?"
            }
            """

        input_messages=[{"role":"system","content":system_prompt}]+messages[-3:]
        chatbot_output=get_chatbot_response(client,input_messages,0.7)
        print(chatbot_output)
        output = postprocess(chatbot_output)
        
        return output
        
        

In [36]:
def postprocess(output):
    try:
        # Remove triple backticks or markdown code block if present
        cleaned_output = output.strip().strip("`").strip()
        parsed_output = json.loads(cleaned_output)
    except Exception as e:
        raise ValueError(f"Error parsing model output: {output}") from e

    return {
        "role": "assistant",
        "content": parsed_output['message'],
        "memory": {
            "agent": "guard_agent",
            "guard_decision": parsed_output['decision']
        }
    }


In [44]:
def get_chatbot_response(client,messages,temperature=0):
    input_messages=[]
    for message in messages:
        input_messages.append({"role":message["role"],"content":message["content"]})

        response=client.invoke(input=input_messages,temperature=temperature)

    return response.content


In [45]:
test_messages = [
    {"role": "user", "content": "Please tell us about the staff?"}
]

response = get_response(test_messages)
print(response)


{
"chain of thought": ["staff", "coffee shop"],
"decision": "not allowed",
"message": "Sorry, I can't help with that. Can I help you with your order?"
}
{'role': 'assistant', 'content': "Sorry, I can't help with that. Can I help you with your order?", 'memory': {'agent': 'guard_agent', 'guard_decision': 'not allowed'}}


In [3]:
from sentence_transformers import SentenceTransformer

In [10]:
from pinecone import Pinecone

In [ ]:
pc=Pinecone(api_key=os.getenv("PINECONE_API_KEY"))



index_name=os.getenv("INDEX_NAME")


In [31]:
def get_closest_result(input_embeddings,index_name,top_k=2):
        index=pc.Index(index_name)

        results=index.query(
            namespace="ns1",
            vector=input_embeddings.tolist(),
            top_k=top_k,
            include_value=False,
            include_metadata=True
        )
        
        return results
    


def get_chatbot_response(client,messages,temperature=0):
    input_messages=[]
    for message in messages:
        input_messages.append({"role":message["role"],"content":message["content"]})

        response=client.invoke(input=input_messages,temperature=temperature)

    return response.content


def get_embedding(model,model_name,user_prompt):
    embedding=model.encode(user_prompt)
    return embedding

In [6]:
load_dotenv()
client=ChatGroq(api_key=os.getenv("GROQ_API_KEY"),model=os.getenv("MODEL_NAME"))
embedding_model=SentenceTransformer(os.getenv("EMBEDDING_MODEL"))

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


In [ ]:
def get_response(self,messages):
        messages = deepcopy(messages)

        user_message = messages[-1]['content']
        embedding = get_embedding(self.embedding_client,self.model_name,user_message)[0]
        result = self.get_closest_results(self.index_name,embedding)
        source_knowledge = "\n".join([x['metadata']['text'].strip()+'\n' for x in result['matches'] ])

        prompt = f"""
        Using the contexts below, answer the query.

        Contexts:
        {source_knowledge}

        Query: {user_message}
        """

        system_prompt = """ You are a customer support agent for a coffee shop called Merry's way. You should answer every question as if you are waiter and provide the neccessary information to the user regarding their orders """
        messages[-1]['content'] = prompt
        input_messages = [{"role": "system", "content": system_prompt}] + messages[-3:]

        chatbot_output =get_chatbot_response(self.client,self.model_name,input_messages)
        output = self.postprocess(chatbot_output)
        return output

In [19]:
model_name=os.getenv("EMBEDDING_MODEL")
user_message="I WANT A CAPPUCINNO"
embedding=get_embedding(embedding_model,model_name,user_message)

In [35]:
result=get_closest_result(embedding,index_name)
result

{'matches': [{'id': 'Cappuccino',
              'metadata': {'text': 'Cappuccino : A rich and creamy cappuccino '
                                   'made with freshly brewed espresso, steamed '
                                   'milk, and a frothy milk cap. This '
                                   'delightful drink offers a perfect balance '
                                   'of bold coffee flavor and smooth milk, '
                                   'making it an ideal companion for relaxing '
                                   'mornings or lively conversations. -- '
                                   "Ingredients: ['Espresso', 'Steamed Milk', "
                                   "'Milk Foam'] -- Price: 4.5 -- rating: 4.7"},
              'score': 0.6152215,
              'values': []},
             {'id': 'Espresso shot',
              'metadata': {'text': 'Espresso shot : A bold shot of rich '
                                   'espresso, our espresso is crafted from the '
     

In [34]:
source_knowledge = "\n".join([x['metadata']['text'].strip()+'\n' for x in result['matches'] ])
source_knowledge

"Cappuccino : A rich and creamy cappuccino made with freshly brewed espresso, steamed milk, and a frothy milk cap. This delightful drink offers a perfect balance of bold coffee flavor and smooth milk, making it an ideal companion for relaxing mornings or lively conversations. -- Ingredients: ['Espresso', 'Steamed Milk', 'Milk Foam'] -- Price: 4.5 -- rating: 4.7\n\nEspresso shot : A bold shot of rich espresso, our espresso is crafted from the finest beans to deliver a robust flavor in every sip. Perfect for a quick pick-me-up, it can also serve as a base for your favorite coffee drinks. -- Ingredients: ['Espresso'] -- Price: 2.0 -- rating: 4.9\n"

In [39]:
text='\n'.join([x['metadata']['text'].strip()+"\n" for x in result['matches']])
print(text)

Cappuccino : A rich and creamy cappuccino made with freshly brewed espresso, steamed milk, and a frothy milk cap. This delightful drink offers a perfect balance of bold coffee flavor and smooth milk, making it an ideal companion for relaxing mornings or lively conversations. -- Ingredients: ['Espresso', 'Steamed Milk', 'Milk Foam'] -- Price: 4.5 -- rating: 4.7

Espresso shot : A bold shot of rich espresso, our espresso is crafted from the finest beans to deliver a robust flavor in every sip. Perfect for a quick pick-me-up, it can also serve as a base for your favorite coffee drinks. -- Ingredients: ['Espresso'] -- Price: 2.0 -- rating: 4.9

